In [5]:
import pandas as pd

In [6]:
import nltk 

In [2]:
nltk.download_shell()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] bcp47............... BCP-47 Language Tags
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [ ] cess_esp.

In [11]:
dst=pd.read_csv('spam.csv',encoding="ISO-8859-1",sep=',')
dst

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [219]:
messages=pd.read_csv('spam.csv', encoding = "ISO-8859-1",sep=',')
messages=messages.iloc[:,0:2]
messages.columns=['label','messages']

# messages.rename(columns={'v1':'label','v2':'messages'},inplace=True)
messages.head()

,label,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [32]:
import string

In [55]:
import nltk 

In [56]:
from nltk.corpus import stopwords 

In [57]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dwany\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [92]:
def clean_mesg(msg):
    nopunc=[i for i in msg if i not in string.punctuation]
    nopunc="".join(nopunc)
    return [i for i in  nopunc.split() if i.lower() not in stopwords.words("english")]

In [209]:
messages['messages'].head(5).apply(clean_mesg)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: messages, dtype: object

In [142]:
from sklearn.feature_extraction.text import CountVectorizer

In [146]:
cv=CountVectorizer(analyzer=clean_mesg).fit(messages['messages'])

In [148]:
print(len(cv.vocabulary_))


11304


In [150]:
msgs=messages['messages'][6]

In [151]:
print(msgs)

Even my brother is not like to speak with me. They treat me like aids patent.


In [152]:
cvtran=cv.transform([msgs])

In [154]:
print(cvtran)

  (0, 1777)	1
  (0, 4512)	1
  (0, 5111)	1
  (0, 7698)	2
  (0, 8653)	1
  (0, 9860)	1
  (0, 10513)	1


In [156]:
cv.get_feature_names()[7698]

C:\Users\dwany\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'like'

In [158]:
mesg_cv=cv.transform(messages['messages'])

In [160]:
print('Shape of Sparse Matrix:',mesg_cv.shape)

Shape of Sparse Matrix: (5572, 11304)


In [161]:
mesg_cv.nnz #num  of non zero elements

50193

In [162]:
#TERM FREQUENCY-INVERSE DOCUMENT FREQUENCY

#difference between TF and DF
#TF is frequency counter for a term t in document d
#DF is the count of occurences of term t in the document set N

In [163]:
from sklearn.feature_extraction.text import TfidfTransformer

In [167]:
tfidf_trans=TfidfTransformer().fit(mesg_cv)
tfidf=tfidf_trans.transform(cvtran)
print(tfidf)

  (0, 10513)	0.33521441416639297
  (0, 9860)	0.32680848361658965
  (0, 8653)	0.43692878744892083
  (0, 7698)	0.4149098236609654
  (0, 5111)	0.3383712690711893
  (0, 4512)	0.43692878744892083
  (0, 1777)	0.33521441416639297


In [224]:
#creating data pipeline svm
from sklearn import svm
from sklearn.metrics import classification_report


In [210]:
from sklearn.model_selection import train_test_split
m_train,m_test,l_train,l_test=train_test_split(messages['messages'],messages['label'],test_size=0.2)

In [220]:
from sklearn.pipeline import Pipeline 
pline=Pipeline([
    ('a',CountVectorizer(analyzer=clean_mesg)),
    ('b',TfidfTransformer()),
    ('c',svm.SVC(C=0.1,kernel='linear',degree=3,gamma='auto')),
])

In [221]:
pline.fit(m_train,l_train)

Pipeline(steps=[('a',
                 CountVectorizer(analyzer=<function clean_mesg at 0x000002044B3758B0>)),
                ('b', TfidfTransformer()),
                ('c', SVC(C=0.1, gamma='auto', kernel='linear'))])

In [216]:
predictions=pline.predict(m_test)

In [222]:
print(classification_report(predictions,l_test))

              precision    recall  f1-score   support

         ham       1.00      0.91      0.95      1077
        spam       0.28      1.00      0.44        38

    accuracy                           0.91      1115
   macro avg       0.64      0.95      0.70      1115
weighted avg       0.98      0.91      0.94      1115

